# RSA cryptosystem
Intuition: it is feasible to find integers $d, e, n$ such that
$$ (m^e)^d\equiv m \pmod{n}$$
for integers $m<n$, such that it is infeasible to find $d$ given $e$, $n$, and/or $m$, and that
$$ (m^d)^e\equiv m \pmod{n}.$$


In [20]:
from Crypto.Util.number import getPrime, inverse, size
import secrets

In [21]:
def new_key(key_bits:int):
    p = getPrime(key_bits//2)
    q = getPrime(key_bits//2)

    # public key:
    n = p*q
    # exponent:
    e = 65537
    
    # totient function
    phi = (p-1)*(q-1)
    # private key:
    d = inverse(e, phi)
    
    return(e, n, d)

In [34]:
def encrypt(plaintext:int, exponent:int, public_key:int) -> int:

    ciphertext = pow(plaintext, exponent, public_key)
    
    return ciphertext

In [35]:
def decrypt(ciphertext:int, private_key:int, public_key:int) -> int:

    plaintext = pow(ciphertext, private_key, public_key)
    
    return plaintext

In [24]:
def new_flag(flag_bytes:int):
    with open('/usr/share/dict/words') as w:
		
        numWords = flag_bytes
        words = [word.strip() for word in w]

        flag = ' '.join(secrets.choice(words) for i in range(numWords))
    return flag[0:flag_bytes]

In [25]:
key_bits = 256
(e,n,d) = new_key(key_bits)
flag_bytes = 30
flag = new_flag(flag_bytes)

# encode the string as an integer
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')
ciphertext = encrypt(f, e, n)

plaintext = decrypt(ciphertext, d, n)
# decode the plaintext as string
p = plaintext.to_bytes(key_bits//8, byteorder='big').decode('ascii')

## Special cases

In [ ]:
print(encrypt(0,e,n))
print(encrypt(1,e,n))
print(encrypt(n,e,n))

## Attacks
---
### Small exponent $e$  - plaintext recovery
Suppose $e=3$. For a small plaintext $m$, it is possible to brute-force guess $m$: $$\sqrt[e]{c}$$

In [40]:
import gmpy2
message = 'secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c = pow(m, 3, n)
print(f'modulus: {n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

modulus: 52483463755050358431291594167767225838979059379795538692642750521578587312439
ciphertext: 2042548109113812252163525474272795974844736
cube root(c): (mpz(126879297332596), mpz(0))
guess: 126879297332596
original m: 126879297332596


If $e$ is small but $m^3>n$, it may be possible to recover $m$ from small multiples: $$\sqrt[e]{c+kn}$$

In [46]:
message = 'supr secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c_before_modulo = pow(m,3)
print(f'pow(m,3) before modulo: (unkown without private key) \n{c_before_modulo}')
c = pow(m, 3, n)
print(f'modulus: \n{n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

for k in range(100):
    guess = int(gmpy2.iroot_rem(c+k*n,3)[0])
    confirmed = pow(guess, 3, n) == c
    if confirmed:
        print(f'match found for k={k}')
        break

pow(m,3) before modulo: (unkown without private key) 
2719439991958273208733653161439045609027040839550764719847986258704299828330816
modulus: 
52483463755050358431291594167767225838979059379795538692642750521578587312439
ciphertext: 42783340450704928737781858882917091239108811181192246523205982103791875396427
cube root(c): (mpz(34975040831413158487309332), mpz(876629958637493208307685442414314476310612321154059))
guess: 34975040831413158487309332
original m: 139581060393214157926851956
match found for k=51


---
## Factoring $n$ - private key recovery
The private key $d$ is constructed from the prime factors of $n$ via the totient function:
$$
\phi(n) = (p-1)\cdot(q-1)\\
d = e^{-1}\pmod{\phi(n)}
$$
Factoring $n$ allows immediate computation of $d$.

Integer factoring options: look-up or computation
- [factordb](http://factordb.com/) or [wolframalpha](https://www.wolframalpha.com/input/?i=factor+)
- [pyecm](https://github.com/martingkelly/pyecm) using [gmpy2](https://github.com/aleaxit/gmpy)

In [10]:
import pyecm
import math

P = getPrime(32)
Q = getPrime(32)
N = P*Q
verbose = False
random_sigma = True
asymptotic_speed = 7
processing_power = 1.0
lf = list(pyecm.factors(N, verbose, random_sigma, asymptotic_speed, processing_power))
print(f'factors of {N}: prod {lf} = {math.prod(lf)}')

factors of 12353035358388743171: prod [mpz(3281841361), mpz(3764056211)] = 12353035358388743171


Factorization difficulty depends on the best known algorithm. For recommendations, see [SP 800-56b](https://csrc.nist.gov/publications/detail/sp/800-56b/rev-2/final).

---
## common factor
Suppose moduli $m,n$ were generated with one prime factor $p$ in common; then $p=\gcd{m,n}$.

$\gcd$ is an extremely quick computation; large keys offer no protection.

See surveys by [Lenstra et al](https://eprint.iacr.org/2012/064) and [Heninger et al.](https://www.usenix.org/conference/usenixsecurity12/technical-sessions/presentation/heninger).

In [18]:
from math import gcd
p = getPrime(2048)
q = getPrime(2048)
r = getPrime(2048)
m = p*q
n = p*r
print(f'gcd(m,n) = \n{gcd(m,n)}')
print(f'check: p = \n{p}')

gcd(m,n) = 
26815956581423103359639388739227375486729327191521064439200612917557997203718666616757319132554692571254242871845056948911524055741393408414547394728532939737063906306374966054809787745097906319910098341727407960065665932461953594734532739281068816094247662586446983966700564071573064794281207090993409366310421449669017375310616218791884204603262015374412502350564937378184614399667905384018091494990034604102596699991805461415642234374511170303138658889725032172951944175723585418771518725562386375279109772853235976421016335356522385658663402742849258298417380050438233360033352679664291994273951617250507168434043
check: p = 
2681595658142310335963938873922737548672932719152106443920061291755799720371866661675731913255469257125424287184505694891152405574139340841454739472853293973706390630637496605480978774509790631991009834172740796006566593246195359473453273928106881609424766258644698396670056407157306479428120709099340936631042144966901737531061621879188420460326201537441250

## blinding
## Hastad broadcast